In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

# 1


In [2]:
titanic = sns.load_dataset('titanic')
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


* a.타이타닉호의 승객에 대해 나이와 성별에 의한 카테고리 열인 category1 열을 만드시오. category1 카테고리는 다음과 같이 정의됨

​

1)20살이 넘으면 성별을 그대로 사용한다.

2)20살 미만이면 성별에 관계없이 “child”라고 한다.

In [3]:
titanic['category1'] = titanic.agg(lambda r : r.sex if r.age >= 20 else 'child', axis = 1)
titanic[['sex','age','category1']].head(10)

,sex,age,category1
0,male,22.0,male
1,female,38.0,female
2,female,26.0,female
3,female,35.0,female
4,male,35.0,male
5,male,NaN,child
6,male,54.0,male
7,male,2.0,child
8,female,27.0,female
9,female,14.0,child


* b.타이타닉호의 승객 중 나이를 명시하지 않은 고객은 나이를 명시한 고객의 평균 나이 값이 되도록 titanic 데이터프레임을 고치시오.

In [4]:
titanic['age'] = titanic['age'].fillna(np.round(titanic.age.mean()))
titanic['age']

0      22.0
1      38.0
2      26.0
3      35.0
4      35.0
       ... 
886    27.0
887    19.0
888    30.0
889    26.0
890    32.0
Name: age, Length: 891, dtype: float64

* c.성별, 선실(class)별, 출발지(embark_town)별 생존율을 구하시오.

In [5]:
titanic.groupby(['sex','class','embarked']).mean()['survived']

sex     class   embarked
female  First   C           0.976744
                Q           1.000000
                S           0.958333
        Second  C           1.000000
                Q           1.000000
                S           0.910448
        Third   C           0.652174
                Q           0.727273
                S           0.375000
male    First   C           0.404762
                Q           0.000000
                S           0.354430
        Second  C           0.200000
                Q           0.000000
                S           0.154639
        Third   C           0.232558
                Q           0.076923
                S           0.128302
Name: survived, dtype: float64

In [6]:
# 이런 것도 가능
pd.crosstab([titanic.sex,titanic.survived],titanic['class'],margins=True)

class            First  Second  Third  All
sex    survived                           
female 0             3       6     72   81
       1            91      70     72  233
male   0            77      91    300  468
       1            45      17     47  109
All                216     184    491  891

* d.타이타닉호 승객을 ‘미성년자’, ‘청년’, ‘중년’, ‘장년’, ‘노년’ 나이 그룹으로 나누고,

각 그룹별 생존율을 구하시오.

In [7]:
bins = [1, 20, 30, 50, 70, 100]
labels = ["미성년자", "청년", "중년", "장년", "노년"]

titanic['age_group'] = pd.cut(titanic.age, bins, labels=labels)

titanic.groupby('age_group').mean()['survived']

age_group
미성년자    0.424242
청년      0.334152
중년      0.423237
장년      0.355932
노년      0.200000
Name: survived, dtype: float64

In [21]:
titanic.pivot_table('survived',"age_group")

,survived
age_group,
미성년자,0.424242
청년,0.334152
중년,0.423237
장년,0.355932
노년,0.200000


* e.qcut 명령으로 세 개의 나이 그룹을 만들고, 나이 그룹별 남녀 성비와 생존율을 구하시오.

In [8]:
titanic['age_group2'] = pd.qcut(titanic.age,3,labels=['group1','group2','group3'])
titanic[['age','age_group2']]

,age,age_group2
0,22.0,group1
1,38.0,group3
2,26.0,group2
3,35.0,group3
4,35.0,group3
...,...,...
886,27.0,group2
887,19.0,group1
888,30.0,group2
889,26.0,group2


In [9]:
titanic_sex = titanic.groupby('age_group2')['sex'].count()
titanic_sex

age_group2
group1    301
group2    304
group3    286
Name: sex, dtype: int64

In [10]:
# 그룹별 남녀성비
titanic.groupby(['age_group2','sex']).count()['survived'] / titanic_sex

age_group2  sex   
group1      female    0.405316
            male      0.594684
group2      female    0.319079
            male      0.680921
group3      female    0.332168
            male      0.667832
dtype: float64

In [11]:
# 그룹별, 성별, 생존율
titanic.groupby(['age_group2','sex']).mean()['survived']

age_group2  sex   
group1      female    0.713115
            male      0.206704
group2      female    0.701031
            male      0.164251
group3      female    0.821053
            male      0.198953
Name: survived, dtype: float64

In [12]:
titanic.groupby(['age_group2','sex']).describe()['survived']['count']/titanic_sex

age_group2  sex   
group1      female    0.405316
            male      0.594684
group2      female    0.319079
            male      0.680921
group3      female    0.332168
            male      0.667832
dtype: float64

# 2

In [13]:
mpg = sns.load_dataset('mpg')
mpg.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino


* a.배기량(displacement) 대비 마력(horsepower) 열(hp_per_cc)을 추가하시오.

In [14]:
mpg['hp_per_cc'] = mpg['horsepower'] / mpg['displacement']
mpg.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name,hp_per_cc
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu,0.423453
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320,0.471429
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite,0.471698
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst,0.493421
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino,0.463576


* b.
name으로부터 manufacturer(제조사)와 모델을 추출하여  
새로운 열 manufacturer와 model을 추가하고, name 열은 삭제하시오.

In [15]:
mpg['manufacturer'] = mpg['name'].str.split().str[0]
mpg['model'] = mpg['name'].str.split().str[1:].apply(' '.join)
mpg.drop(['name'],inplace=True,axis=1)
mpg.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,hp_per_cc,manufacturer,model
0,18.0,8,307.0,130.0,3504,12.0,70,usa,0.423453,chevrolet,chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,0.471429,buick,skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,0.471698,plymouth,satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,0.493421,amc,rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,0.463576,ford,torino


* c.엔진의 실린더(cylinders) 갯수별 연비(mpg)의 평균을 구하시오.

In [16]:
mpg.groupby('cylinders').mean()['mpg']

cylinders
3    20.550000
4    29.286765
5    27.366667
6    19.985714
8    14.963107
Name: mpg, dtype: float64

In [23]:
mpg[['cylinders','mpg']].groupby('cylinders').mean()

,mpg
cylinders,
3,20.550000
4,29.286765
5,27.366667
6,19.985714
8,14.963107


In [27]:
mpg.pivot_table('mpg','cylinders',aggfunc='mean')

,mpg
cylinders,
3,20.550000
4,29.286765
5,27.366667
6,19.985714
8,14.963107


* d.생산지(origin)별 배기량 대비 마력(hp_per_cc)의 평균을 구하시오.

In [17]:
mpg.groupby('origin').mean()['hp_per_cc']

origin
europe    0.740005
japan     0.789485
usa       0.512063
Name: hp_per_cc, dtype: float64

In [28]:
mpg.pivot_table("hp_per_cc",'origin',aggfunc='mean')

,hp_per_cc
origin,
europe,0.740005
japan,0.789485
usa,0.512063


* e.모델이 5개 이상인 제조사에 대하여  
연비(mpg)의 평균이 가장 좋은 제조사 Top 5를 구하시오.


In [18]:
mpg_filtered = mpg.groupby('manufacturer').filter(lambda x : x['manufacturer'].count() >= 5)        # 그룹바이 이후 filtering => SQL의 having 절절

In [19]:
mpg_filtered.groupby('manufacturer').mean()['mpg'].sort_values(ascending=False)[:5]

manufacturer
vw         39.016667
honda      33.761538
renault    32.880000
datsun     31.113043
mazda      30.860000
Name: mpg, dtype: float64

In [41]:
mpg_filtered.pivot_table('mpg','manufacturer').sort_values(by='mpg',ascending=False)[:5]

,mpg
manufacturer,
vw,39.016667
honda,33.761538
renault,32.880000
datsun,31.113043
mazda,30.860000


In [31]:
# 다른 방법
# 제조사별 value찾기
s = mpg.manufacturer.value_counts()
s

ford             51
chevrolet        43
plymouth         31
amc              28
dodge            28
toyota           25
datsun           23
buick            17
pontiac          16
volkswagen       15
honda            13
mercury          11
oldsmobile       10
mazda            10
peugeot           8
fiat              8
audi              7
volvo             6
chrysler          6
vw                6
renault           5
opel              4
subaru            4
saab              4
chevy             3
maxda             2
cadillac          2
bmw               2
mercedes-benz     2
toyouta           1
triumph           1
vokswagen         1
capri             1
mercedes          1
hi                1
chevroelt         1
nissan            1
Name: manufacturer, dtype: int64

In [32]:
big = s[s>=5].index.tolist()
print(big)

['ford', 'chevrolet', 'plymouth', 'amc', 'dodge', 'toyota', 'datsun', 'buick', 'pontiac', 'volkswagen', 'honda', 'mercury', 'oldsmobile', 'mazda', 'peugeot', 'fiat', 'audi', 'volvo', 'chrysler', 'vw', 'renault']


In [38]:
# R 의 in 연산자 대신에 isin() 매소드 사용
# 모델이 5개 이상인 제조사들을 필터링한 결과
big_mpg = mpg[mpg.manufacturer.isin(big)]
big_mpg.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,hp_per_cc,manufacturer,model
0,18.0,8,307.0,130.0,3504,12.0,70,usa,0.423453,chevrolet,chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,0.471429,buick,skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,0.471698,plymouth,satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,0.493421,amc,rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,0.463576,ford,torino


In [40]:
big_mpg.pivot_table('mpg','manufacturer').sort_values(by='mpg',ascending=False)[:5]

,mpg
manufacturer,
vw,39.016667
honda,33.761538
renault,32.880000
datsun,31.113043
mazda,30.860000
